In [14]:
import pandas as pd

nutritional_values = pd.read_csv("../data/nutritional_values.csv")
recipes = pd.read_csv("../data/filtered_recipes_100.csv", sep="|")
ingredients = pd.read_csv("../data/ingredients_100.csv")



recipes_nutritional_values = pd.merge(recipes,
                                         nutritional_values,
                                         on='Recipe_id', 
                                         how='right')



In [15]:
recipes_nutritional_values.head()

,Recipe_id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,...,calcio,"hierro, total",potasio,magnesio,sodio,fósforo,ioduro,"selenio, total",zinc (cinc),Total_Grams
0,1,Cócteles y bebidas,Absolut tequila,3.9,baja,1.0,15m,NaN,https://www.recetasgratis.net/receta-de-absolu...,1,...,0.002240,0.000037,0.035810,0.002320,0.004270,0.003190,0.000001,0.000000,0.000000,42.00000
1,5,Recetas de Verduras,Receta de aguacate y plátano,3.6,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-aguaca...,4,...,0.035885,0.000524,0.178044,0.031209,0.008544,0.040281,0.000003,0.000001,0.000351,58.33445
2,8,Cócteles y bebidas,Cóctel refrescante de menta,2.3,baja,1.0,15m,NaN,https://www.recetasgratis.net/receta-de-coctel...,0,...,0.122750,0.004634,0.260313,0.039056,0.009669,0.051344,0.000003,0.000001,0.000601,210.62500
3,10,Cócteles y bebidas,Batido de yogur y plátanos,3.7,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-batido...,0,...,0.880000,0.002290,2.259500,0.197200,0.394500,0.744000,0.000042,0.000011,0.003215,1100.00000
4,27,Recetas de Pescado,Receta de aperitivo de atún,4.0,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-aperit...,4,...,0.083450,0.002660,0.555276,0.045692,1.049754,0.180909,0.000010,0.000031,0.001117,404.92792


In [138]:
def get_food_percentage(tdee):
    
    breakfast = tdee * 0.2
    snack_1   = tdee * 0.1
    lunch     = tdee * 0.35
    snack_2   = tdee * 0.1
    dinner    = tdee * 0.25
    
    return breakfast, snack_1, lunch, snack_2, dinner

DINNER_CAT = ['Cena']
LUNCH_CAT = ['Plato principal','Entrante','Acompañamiento']
SNACK_CAT = ['Merienda','Cumpleaños','Postre']
BREAKFAST_CAT = ['Desayuno']

categories={'Desayuno': BREAKFAST_CAT,
                    'Comida': LUNCH_CAT,
                    'Snack': SNACK_CAT,
                    'Cena': DINNER_CAT
                    }

 #RANGO DE PORCENTAJES DE CARBOS,PROTEÍNAS Y GRASAS
    
GAIN = [0.40,0.60,0.25,0.35,0.15,0.25]
MAINTAIN = [0.30,0.50,0.25,0.35,0.25,0.35]
LOSS = [0.10,0.30,0.40,0.50,0.30,0.40]
    
objetivos={
    0 : GAIN,
    1 : MAINTAIN,
    2 : LOSS
        }

macros_cals = [4,4,4,4,9,9] #CALORIAS POR GRAMO DE MACRO

def get_food(category, calories):
    
    global total_protes,total_grasas,total_carbos
    
        
    selected_category = recipes_nutritional_values[(recipes_nutritional_values['Tipo'].isin(categories[category])) 
                                    & (recipes_nutritional_values["energia"] > (calories*0.9) )
                                     & (recipes_nutritional_values["energia"] < (calories * 1.1))]
    
        
    selected_recipe = selected_category[['Recipe_id', 'Nombre', 'energia','carbohidratos','proteina','grasa']].sample(n=1)
    
    recipe_name = selected_recipe.iloc[0]['Nombre']
    recipe_id = selected_recipe.iloc[0]['Recipe_id']
    recipe_calories = selected_recipe.iloc[0]['energia']
    
    total_carbos += selected_recipe.iloc[0]['carbohidratos']
    total_protes += selected_recipe.iloc[0]['proteina']
    total_grasas += selected_recipe.iloc[0]['grasa']

    print("Carbos: " + str(round((total_carbos),1)) + ", Proteinas: " 
          + str(round((total_protes),1)) + ", Grasas: " + str(round((total_grasas),1)) )
    
    recipe = {
        'Nombre':selected_recipe.iloc[0]['Nombre'],
        'Id': selected_recipe.iloc[0]['Recipe_id'],
        'Calorias': selected_recipe.iloc[0]['energia']} 
    
    return recipe_id, recipe_name, recipe_calories


def get_macros(objective,tdee):
 
    pctg_array = objetivos.get(objective)  #PORCENTAJE DE CALORÍAS EN CADA MACRO
    
    cals_x_macro = [i * tdee for i in pctg_array] #CALORÍAS MÍNIMOS Y MÁXIMOS PARA CADA MACRO
        
    macros_array = [round((i / j),2) for i, j in zip(cals_x_macro, macros_cals)] #GRAMOS DE CADA MACRO
    
    print(macros_array)
        
    
    return macros_array
    


def get_recommendations(tdee, macros_array):
    
    global total_protes,total_grasas,total_carbos
    
    min_carbos = macros_array[0]
    max_carbos = macros_array[1]
    min_protes = macros_array[2]
    max_protes = macros_array[3]
    min_grasas = macros_array[4]
    max_grasas = macros_array[5]
    
    breakfast_cal, snack_1_cal, lunch_cal, snack_2_cal, dinner_cal = get_food_percentage(tdee)
    
    breakfast = get_food('Desayuno',breakfast_cal)
    
    snack_1 = get_food('Snack',snack_1_cal)

    lunch = get_food('Comida',lunch_cal)

    snack_2 = get_food('Snack',snack_2_cal)

    dinner = get_food('Cena', dinner_cal)
                                                                                                                            
                                                            
    print("*** PLAN DE COMIDAS ***")
    print("DESAYUNO: ", breakfast)
    print("SNACK_1: ", snack_1 )                                                                                                                        
    print("COMIDA: ", lunch )
    print("SNACK_2: ", snack_2 )                                                                                                                        
    print("CENA: ", dinner )

    
    if int(total_carbos) in range(int(min_carbos),int(max_carbos)) and int(total_protes) in range(int(min_protes),int(max_protes)) and int(total_grasas) in range(int(min_grasas),int(max_grasas)):
       print("Cumple macros")
    
    return breakfast, snack_1, lunch, snack_2, dinner                                                                                                               
                                                                                                                            
                                                                                                                 
                   

In [139]:
tdee_sample = 2500
objective = 1 # mantain
macros_array = get_macros(objective,tdee_sample) #GRAMOS MÍN Y MÁX DE CADA MACRO, CARBOS, PROTEÍNAS, GRASAS
                    


[187.5, 312.5, 156.25, 218.75, 69.44, 97.22]


In [142]:
global total_protes,total_grasas,total_carbos
total_protes,total_grasas,total_carbos = 0,0,0

breakfast, snack_1, lunch, snack_2, dinner = get_recommendations(tdee_sample,macros_array)

Carbos: 62.2, Proteinas: 28.5, Grasas: 18.9
Carbos: 75.2, Proteinas: 77.6, Grasas: 19.4
Carbos: 177.1, Proteinas: 154.6, Grasas: 45.2
Carbos: 239.7, Proteinas: 158.7, Grasas: 45.5
Carbos: 266.9, Proteinas: 206.9, Grasas: 80.0
*** PLAN DE COMIDAS ***
DESAYUNO:  (8024, 'Receta de revuelto de champiñones', 538.21)
SNACK_1:  (51707, 'Receta de gelatinas de colores', 258.00487999999996)
COMIDA:  (68114, 'Receta de espirales con setas y gambas (sin gluten)', 958.170544)
SNACK_2:  (59971, 'Receta de postre de granada y yogur', 274.93)
CENA:  (69428, 'Receta de tortillas de maíz rellenas de jamón y queso', 614.58)
Cumple macros
